In [1]:
import numpy as np
import pandas as pd
import pymongo
import bson
from tqdm import tqdm
from bson.objectid import ObjectId
import operator
from random import shuffle

In [2]:
class MongoServer():
    credentials = None
    conn = None
    db = None
    collections = {}
    def __init__(self, credentials, run = False, db_name = "agile_data_science_group_3"):
        if run:
            if not self.connect2Mongo(credentials, db_name):
                print("Connection to server Failed.")
            
            if not self.connect2DataBase(db_name):
                print("Connection Data Base Failed.")
    """Rotine to connect to Mongo DB"""    
    def connect2Mongo(self, credentials, db):
        try:
            #use your database name, user and password here:
            name,password,url,dbname = credentials['name'], credentials['password'], credentials['url'], credentials['dbname']
            conn=pymongo.MongoClient("mongodb://{}:{}@{}/{}".format(name,password,url,dbname))
            self.conn = conn
            return True
        except pymongo.errors.ConnectionFailure as e:
            return False
    
    """Routine to connect to a Data Base"""
    def connect2DataBase(self, db_name = "agile_data_science_group_3"):
        try:
            self.db = self.conn[db_name]
            return True
        except:
            return False
    
    """Return the available collections in a data base"""
    def listOfCollections (self):
        return self.db.collection_names()
    
    """Donwload all the ollections from the data base"""
    def getAllCollections (self):
        collections = self.listOfCollections()
        for col in collections:
            self.getCollectionFromServer(col)
        return True
    
    
    """Routine to get one collection of the Data Base"""
    # return the collection
    def getCollectionFromServer (self, name_collection):
        if name_collection in self.db.collection_names():
            self.collections[name_collection] = self.db.get_collection(name_collection)
            print ("Collection ",name_collection," Update in Local.")
            return True
        return False
    
    """ Take the collection from the local copy"""
    def getCollection(self, name_collection):
        if not name_collection in self.collections:
            if not self.getCollectionFromServer(name_collection):
                return False
        return self.collections[name_collection]
    
    """ Gets all the items of the collection"""
    def getItems(self, name_collection, N = None):
        if not name_collection in self.collections:
            if not self.getCollectionFromServer(name_collection):
                return False
        if N != None:
            return [element for element in self.collections[name_collection].find().limit(N)]
        else:
            return [element for element in self.collections[name_collection].find()]
    
    """Query in one Collection"""
    def searchInCollection(self, name_collection, field, patro, N = None):
        if N == None:
            return [element for element in self.db.get_collection(name_collection).find({field:patro})]
        else:
            return [element for element in self.db.get_collection(name_collection).find({field:patro}).limit(N)]
    
    """Query in all the Collections"""
    def searchInDB (self, field, patro):
        query = {}
        for collection in self.db.collection_names():
            query[collection] = [element for element in self.db.get_collection(collection).find({field:patro})]
        return query
    
    """Find one in the collection"""
    def findOne(self, collection_name):
        return self.db.get_collection(collection_name).find_one()
    
    """Seach in collection with multiple querys"""
    def searchWithMultiplyConditions(self, collection_name, _query, condition ='$and', N = 6, skip = 0):
        query = []
        for item in self.db.get_collection(collection_name).find({condition:_query}).limit( N + skip):
            query.append(item)
        return query[skip:]
             
    """Find N elements in one collection"""
    def findNElement(self, collection_name, N):
        query = []
        for item in self.db.get_collection(collection_name).find().limit( N ):
            query.append(item)
        return query
    
    """Insert one element into collection"""
    def insertInCollection (self, collection_name,  item):
        assert type(item) == dict, "Item must be a dictionary"
        return self.db.get_collection(collection_name).insert_one(item)
     
        
MS = MongoServer({'name':'huang', 'password':'chen1992', 'url':'ds233895.mlab.com:33895', 'dbname': 'agile_data_science_group_3'}, True)

In [12]:
class Recommender:
    
    def __init__(self, credentials):
        # connect to mongo with MongoServer object
        self.server = MongoServer(credentials, True)
        
    """Dummie Recommender"""
    def dummieRecommendation(self, N = 10):
        listObjectIds = []
        for item in self.server.findNElement('recipes', 1000):
            listObjectIds.append(item['_id'])
        shuffle(listObjectIds)
        return listObjectIds[:N]
        
    """Method that check if the object is a ObjectId"""
    def isObjectId(self, _id):
        try:
            # Do a query in a specific user collection
            if not type(_id) == bson.objectid.ObjectId:
                # creation of a objectID
                if type(_id) == str:
                    idUser = bson.objectid.ObjectId(_id)
                else:
                    return None
            return _id
        except:
            return None
            
    """Search user by idRecepie(ObjectId)""" 
    def searchRecepieWithIngredientsByIs(self, idRecepie):
        idRecepie = self.isObjectId(idRecepie)
        if idRecepie == None:
            print ("idRecepie is not a ObjectId")
            return []
        
        # Search the recepie    
        _collection = 'recipes_ingredients'
        _field = '_id' # ObjectId
        _patro = idRecepie
        query = self.server.searchInCollection(name_collection = _collection, field = _field, patro = _patro)[0]    
        # obtain the ingredients
        return query, query.values()[1]
    

    """Search user by idUser(ObjectId)"""
    def searchUsersById(self, idUser):
        idUser = self.isObjectId(idUser)
        if idUser == None:
            print ("Id User is not a ObjectId")
            return []
        
        _collection = 'users'
        _field = '_id' # ObjectId
        _patro = idUser
        query = self.server.searchInCollection(name_collection = _collection, field = _field, patro = _patro)[0]
        
        return query
    
    """Method that compute a smaller matrix for BestRated"""
    def computeRecommenderMatrixBestRated(self, idRecipe):
        # look the ingredients used in the recipe
        # look the recipe that use on of the ingridients
        # compute Recommender Matrix 
        return None
    
    """Method that compute a smaller matrix for collaborativeFiltering"""
    def computeRecommenderMatrixCollaborativeFiltering(self, idUser, n=10):
        m_ids = []
        m_user = []
        
        idUser = self.isObjectId(idUser)
        if idUser == None:
            print ("Id User is not a ObjectId")
            return []
        
        # look for the ratings of the user
        ratingsUser = self.server.searchInCollection(name_collection='ratings', field='user_id', patro=idUser, N=n) 
        if ratingsUser == []:
            print ("User has no ratings. Cold Start.")
            return None
        ratings = []
        m_user.append(idUser) 
        
        # obtain ratings from the same recipes
        for rating in tqdm(ratingsUser):
            m_ids.append(rating['recipe_id'])
            # search for more ratings in the same recipe
            recipes = self.server.searchInCollection(name_collection='ratings', field='recipe_id', patro=rating['recipe_id'])
            
            # acumulate the ratings
            ratings += recipes
            
            # look if objectId of recipe is in the list
            for recipe in recipes:
                if not recipe['recipe_id'] in m_ids:
                    m_ids.append(recipe['recipe_id'])
                    
                if not recipe['user_id'] in m_user:
                    m_user.append(recipe['user_id'])

        # compute Recommender Matrix 
        matrix = pd.DataFrame(np.full((len(m_ids), len(m_user)), np.nan), index=m_ids, columns=m_user)
        for rates in ratings:
            matrix[rates['user_id']][rates['recipe_id']] = rates['rating']
        
        return matrix
    
    """Method that donwliad the matrix from ratings and generates this one"""
    def generateRatingMatrix(self):
        m_ids = set()
        m_user = set()
        ratings = self.server.getItems('ratings')
        for item in ratings:
            m_ids.add(item["recipe_id"])
            m_user.add(item["user_id"])
        
        matrix = pd.DataFrame(np.full((len(m_ids), len(m_user)), np.nan), index=m_ids, columns=m_user)
        for rates in ratings:
            matrix[rates['user_id']][rates['recipe_id']] = rates['rating']
        
        return matrix
            
    def computeRecomendation(self):
        # Introduce here the distance function for each of the cases
        # maybe it is necesary to separete the funciton in two
        pass
    
   
    """Recommender of Collaborative Filtering"""
    def collaborativeFiltering(self, idUser, N = 6, skip = 0):
        # take the ratings of the user
        # with the recipes of the user, find which recepes we can generate
        # generate the recommender matrix for user
        # call the distance function
        return self.server.searchInCollection('CollRecom', 'user_id', idUser,  N = N + skip)[0]['recomendations'][skip:skip+N]
    
    """ Method that search in function of the ingredients"""
    def searchRecepieByIngredients(self, listIngredients, N = 6, skip = 0):
        query = []
        for ingredient in listIngredients:
            query.append({'ingredients':ingredient})
        
        respons = self.server.searchWithMultiplyConditions('RecIng', query, N = N, skip = skip)
        
        objectsIds = []
        for recepie in respons:
            objectsIds.append(recepie['recipe_id'])
            
        return objectsIds
    
    """ Return top n recipes by maximum mean rating. In case of draw, then by minimum standard deviation rating. """
    def bestRatedWeb(self, n=10):
        data = pd.DataFrame(self.server.getItems('ratings', N = 2000))
        # top rated
        data["rating"] = data["rating"].astype(float)
        recipe_rating_mean = data.groupby(['recipe_id'])['rating'].mean()
        recipe_rating_std = data.groupby(['recipe_id'])['rating'].std()
        recipe_rating = pd.concat([recipe_rating_mean, recipe_rating_std], axis=1)
        recipe_rating.columns=["mean", "std"]
        recs = recipe_rating.sort_values(["mean", "std"], ascending=[0,1])
        
        return list(map(ObjectId, list(recs.index.values)[:int(n)]))
    
    def distance_recipes(self, ing1, ing2):
        rec1 = set(ing1)
        rec2 = set(ing2)
        d = len(rec1.intersection(rec2))/len(rec1)
        if d == 1.0:
            return 0
        return d

    """Recommender based on content"""
    def bestRated(self, idRecepie, N = 6, skip = 0):
        recipes_dict = self.server.getItems('RecIng', N = 2000)
        ingridents = self.server.searchInCollection('RecIng', 'recipe_id', idRecepie)[0]['ingredients']

        dis = dict()
        for rec in recipes_dict:
            dis[rec['recipe_id']] = self.distance_recipes(ingridents, rec['ingredients'])

        df_return = sorted(dis.items(), key=operator.itemgetter(1), reverse=True)[skip:N+ skip]

        return [obj for obj, rat in df_return]
    
rec = Recommender({'name':'huang', 'password':'chen1992', 'url':'ds233895.mlab.com:33895', 'dbname': 'agile_data_science_group_3'})

#rec.searchRecepieByIngredients(['oil', 'cold', 'water'], 10)
#rec.computeRecommenderMatrixCollaborativeFiltering(bson.objectid.ObjectId('5a1aa2cb2cfaa80d6ff251f3'))
#rec.generateRatingMatrix()
"""
print(rec.bestRated(objectId, N = 4, skip = 0))
print(rec.bestRated(objectId, N = 2, skip = 0))
print(rec.bestRated(objectId, N = 2, skip = 2))



print(rec.searchRecepieByIngredients(['oil', 'water'], N = 4, skip = 0))
print(rec.searchRecepieByIngredients(['oil', 'water'], N = 2, skip = 0))
print(rec.searchRecepieByIngredients(['oil', 'water'], N = 2, skip = 2))
"""

#rec.computeRecommenderMatrixCollaborativeFiltering(rec.server.findOne('users')['_id'], n = 20)
#rec.server.searchInCollection('users', 'user_name', 'Amin')
#rec.server.findOne('users')
rec.collaborativeFiltering(MS.findOne('users')['_id'])
rec.collaborativeFiltering(MS.findOne('users')['_id'], 6, 5)

[ObjectId('5a073c312cfaa84d14995092'),
 ObjectId('5a073c312cfaa84d14994c83'),
 ObjectId('5a073c312cfaa84d14995758'),
 ObjectId('5a073c312cfaa84d14993958'),
 ObjectId('5a073c312cfaa84d14996a77'),
 ObjectId('5a073c312cfaa84d14994c09')]

In [83]:
MS.findOne('users')

{'_id': ObjectId('5a1aa2cb2cfaa80d6ff251f3'),
 'email': 'Chauncey.Steffen@email.com',
 'password': 'Chauncey',
 'user_name': 'Chauncey Steffen'}

In [76]:
for item in dic:
    MS.insertInCollection('CollRecom', {'user_id':item,'recomendations': dic[item]})

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:105: DeprecationWarning: insert is deprecated. Use insert_one or insert_many instead.


In [61]:
MS.findOne('ratings')

{'_id': ObjectId('5a30678590ea863de40f8d0e'),
 'rating': 2,
 'recipe_id': ObjectId('59fb0d77eee3642f3c0684a4'),
 'user_id': ObjectId('5a1aa2cb2cfaa80d6ff251f3')}

In [68]:
MS.searchInCollection('users', '_id', t(dic))

[{'_id': ObjectId('5a1aa2cb2cfaa80d6ff251f3'),
  'email': 'Chauncey.Steffen@email.com',
  'password': 'Chauncey',
  'user_name': 'Chauncey Steffen'}]

In [67]:
def t (dic):
    for item in dic:
        return item
t(dic)

ObjectId('5a1aa2cb2cfaa80d6ff251f3')

In [77]:
MS.listOfCollections()

['test',
 'users',
 'recipes',
 'ingredients',
 'RecIng',
 'system.indexes',
 'CollRecom',
 'recipes_ingredients',
 'ratings']

In [72]:
MS.searchInCollection('recipes', '_id', dic[t(dic)][0])

[{'_id': ObjectId('5a073c312cfaa84d14993933'),
  'id': 113241,
  'image': 'http://www.seriouseats.com/recipes/assets_c/2010/08/20100831-nb-hp-thumb-625xauto-109011.jpg',
  'list_ingredients': ['1 sheet of dried pig skin(about 5 by 5 inches)',
   '6 cups best-quality pork or vegetable broth',
   'a few leaves napa cabbage(bok choy)(or leafy green vegetable of your choice)',
   'thinly sliced green onions(to garnish)'],
  'name': 'The Nasty Bits: Pork Skin Soup',
  'ningredients': 4.0,
  'serves': 'four as a soup course in a meal',
  'time_min': 'overnight',
  'url': 'http://www.seriouseats.com/recipes/2010/08/the-nasty-bits-pork-skin-soup-recipe.html'}]

In [78]:
MS.findOne('CollRecom')

{'_id': ObjectId('5a329f2390ea86197765c76e'),
 'recomendations': [ObjectId('5a073c312cfaa84d14993933'),
  ObjectId('5a073c312cfaa84d14994fae'),
  ObjectId('5a073c312cfaa84d14995bd8'),
  ObjectId('59fb0d77eee3642f3c068685'),
  ObjectId('5a073c312cfaa84d14993aa0'),
  ObjectId('5a073c312cfaa84d14995092'),
  ObjectId('5a073c312cfaa84d14994c83'),
  ObjectId('5a073c312cfaa84d14995758'),
  ObjectId('5a073c312cfaa84d14993958'),
  ObjectId('5a073c312cfaa84d14996a77'),
  ObjectId('5a073c312cfaa84d14994c09'),
  ObjectId('5a073c312cfaa84d149953f0'),
  ObjectId('5a073c312cfaa84d14996080'),
  ObjectId('5a073c312cfaa84d149939a4'),
  ObjectId('5a073c312cfaa84d14993a58'),
  ObjectId('5a073c312cfaa84d149951c8'),
  ObjectId('5a073c312cfaa84d149964b2'),
  ObjectId('5a073c312cfaa84d14995366'),
  ObjectId('5a073c312cfaa84d1499560f'),
  ObjectId('5a073c312cfaa84d14996063'),
  ObjectId('5a073c312cfaa84d1499430c'),
  ObjectId('5a073c312cfaa84d14994fa8'),
  ObjectId('5a073c312cfaa84d14994d7d'),
  ObjectId('5a07